In [1]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

import torch

import torch.nn as nn
from torch.autograd import Function, Variable
from torch.nn.parameter import Parameter
import torch.nn.functional as F

from qpth.qp import QPFunction
import qpth

import matplotlib as mpl
import matplotlib.pyplot as plt
plt.style.use('bmh')
%matplotlib inline
print(torch.__version__)

1.7.1+cpu


In [2]:
class IneqOptNet(nn.Module):
    def __init__(self, nFeatures, nHidden, nCls, bn, nineq=2, neq=0, eps=1e-4):
        super().__init__()
        self.nFeatures = nFeatures
        self.nHidden = nHidden
        self.bn = bn
        self.nCls = nCls
        self.nineq = nineq
        self.neq = neq
        self.eps = eps

        self.fc1 = nn.Linear(nFeatures, nHidden)
        self.fc2 = nn.Linear(nHidden, nineq * nCls + nineq)

    def forward(self, x):
        nBatch = x.size(0)
        # Normal FC network.
        x = x.view(nBatch, -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        print(x)
        Q = Variable(torch.stack([torch.eye(self.nCls) for _ in range(nBatch)]))

        q = Variable(torch.ones(self.nCls))
        x_t = x[:, :self.nineq * self.nCls]
        A = x_t.reshape(nBatch, self.nineq, self.nCls)
        x_b = x[:, self.nineq * self.nCls:]
        b = x_b
        e = Variable(torch.Tensor())
        x = QPFunction(verbose=1, solver=qpth.qp.QPSolvers.CVXPY)(Q, q, A, b, e, e)
        print(A)
        print(b)
        return F.log_softmax(x)

In [3]:
import math
import torch.optim as optim
import numpy as np


torch.manual_seed(3)
nBatch, nFeatures, nHidden, nCls = 16, 20, 20, 2
# Create random data
x = Variable(torch.randn(nBatch, nFeatures), requires_grad=False)
y = Variable((torch.rand(nBatch) < 0.5).long(), requires_grad=False)

model = IneqOptNet(nFeatures, nHidden, nCls, nineq=2, bn=False)
loss_fn = torch.nn.CrossEntropyLoss()

# Initialize the optimizer.
learning_rate = 1e-3
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

TRAIN_ITER = 10
ATTACK_ITER = 1
PRINT_FREQ = 10

for t in range(TRAIN_ITER):
    # Forward pass: compute predicted y by passing x to the model.
    y_pred = model(x)

    # Compute and print loss.
    loss = loss_fn(y_pred, y)
    if t % PRINT_FREQ == 0:
        print(t, loss.item())

    # Before the backward pass, use the optimizer object to zero all of the
    # gradients for the variables it will update (which are the learnable weights
    # of the model)
    optimizer.zero_grad()

    # Backward pass: compute gradient of the zloss with respect to model
    # parameters
    loss.backward()

    # Calling the step function on an Optimizer makes an update to its
    # parameters
    optimizer.step()


tensor([[ 0.1808,  0.2913,  0.1416,  0.0248,  0.1521,  0.0130],
        [ 0.1084, -0.1917,  0.1003, -0.2235, -0.1771, -0.1979],
        [-0.0273,  0.0112, -0.2736, -0.0182,  0.0333, -0.1634],
        [ 0.2419, -0.4293,  0.0379, -0.0152,  0.0023, -0.1519],
        [ 0.0153, -0.1340, -0.0781, -0.2366, -0.0497, -0.2036],
        [ 0.4946, -0.5082,  0.1141, -0.2986, -0.1932, -0.3327],
        [ 0.0057, -0.4614,  0.1078, -0.3349, -0.0447, -0.1574],
        [-0.4541,  0.6345, -0.3797,  0.0798,  0.3524, -0.4433],
        [ 0.0145,  0.1634, -0.1191, -0.1231,  0.1677, -0.2801],
        [-0.6439, -0.0282, -0.3218, -0.1881,  0.4400, -0.0426],
        [-0.1667,  0.3048, -0.3473,  0.0517, -0.2203, -0.6293],
        [ 0.1353,  0.0685, -0.0170, -0.0046, -0.2843, -0.2416],
        [-0.2131,  0.1217, -0.2505, -0.3916, -0.3869, -0.5044],
        [ 0.0437,  0.1728, -0.0638, -0.0376,  0.1061,  0.0215],
        [ 0.1870, -0.2128,  0.0846, -0.3467,  0.0068,  0.0472],
        [ 0.0654, -0.0664,  0.2500,  0.1

\anaconda3\envs\layers\lib\site-packages\cvxpy\expressions\expression.py:550: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.

  warnings.warn(__STAR_MATMUL_WARNING__, UserWarning)
\anaconda3\envs\layers\lib\site-packages\cvxpy\expressions\expression.py:550: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.

  warnings.warn(__STAR_MATMUL_WARNING__, UserWarning)
\anaconda3\envs\layers\lib\site-packages\cvxpy\expressions\expression.py

variables n = 4, constraints m = 4
          nnz(P) + nnz(A) = 10
settings: linear system solver = qdldl,
          eps_abs = 1.0e-05, eps_rel = 1.0e-05,
          eps_prim_inf = 1.0e-04, eps_dual_inf = 1.0e-04,
          rho = 1.00e-01 (adaptive),
          sigma = 1.00e-06, alpha = 1.60, max_iter = 10000
          check_termination: on (interval 25),
          scaling: on, scaled_termination: off
          warm start: on, polish: on, time_limit: off

iter   objective    pri res    dua res    rho        time
   1  -6.4401e-01   2.84e-01   2.84e+01   1.00e-01   5.39e-05s
  75   2.9442e+03   6.17e-11   2.42e-09   1.15e+05   1.28e-04s

status:               solved
solution polish:      unsuccessful
number of iterations: 75
optimal objective:    2944.1964
run time:             2.36e-04s
optimal rho estimate: 1.46e+05

-----------------------------------------------------------------
           OSQP v0.6.0  -  Operator Splitting QP Solver
              (c) Bartolomeo Stellato,  Goran Banja

\anaconda3\envs\layers\lib\site-packages\cvxpy\expressions\expression.py:550: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.

  warnings.warn(__STAR_MATMUL_WARNING__, UserWarning)
\anaconda3\envs\layers\lib\site-packages\cvxpy\expressions\expression.py:550: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.

  warnings.warn(__STAR_MATMUL_WARNING__, UserWarning)
\anaconda3\envs\layers\lib\site-packages\cvxpy\expressions\expression.py

           OSQP v0.6.0  -  Operator Splitting QP Solver
              (c) Bartolomeo Stellato,  Goran Banjac
        University of Oxford  -  Stanford University 2019
-----------------------------------------------------------------
problem:  variables n = 4, constraints m = 4
          nnz(P) + nnz(A) = 10
settings: linear system solver = qdldl,
          eps_abs = 1.0e-05, eps_rel = 1.0e-05,
          eps_prim_inf = 1.0e-04, eps_dual_inf = 1.0e-04,
          rho = 1.00e-01 (adaptive),
          sigma = 1.00e-06, alpha = 1.60, max_iter = 10000
          check_termination: on (interval 25),
          scaling: on, scaled_termination: off
          warm start: on, polish: on, time_limit: off

iter   objective    pri res    dua res    rho        time
   1  -6.4234e-01   1.98e-01   1.92e+01   1.00e-01   1.11e-03s
  75  -1.6544e-01   3.41e-11   6.02e-10   1.14e+01   2.65e-03s
plsh  -1.6544e-01   2.78e-17   1.11e-16   --------   4.20e-03s

status:               solved
solution polish:      s

...

time_limit: off

iter   objective    pri res    dua res    rho        time
   1  -6.6012e-01   1.57e-01   1.57e+01   1.00e-01   1.21e-04s
  50  -1.0000e+00   1.30e-11   8.17e-11   1.00e-01   7.09e-04s
plsh  -1.0000e+00   0.00e+00   0.00e+00   --------   6.12e-03s

status:               solved
solution polish:      successful
number of iterations: 50
optimal objective:    -1.0000
run time:             6.12e-03s
optimal rho estimate: 3.46e-02

-----------------------------------------------------------------
           OSQP v0.6.0  -  Operator Splitting QP Solver
              (c) Bartolomeo Stellato,  Goran Banjac
        University of Oxford  -  Stanford University 2019
-----------------------------------------------------------------
problem:  variables n = 4, constraints m = 4
          nnz(P) + nnz(A) = 10
settings: linear system solver = qdldl,
          eps_abs = 1.0e-05, eps_rel = 1.0e-05,
          eps_prim_inf = 1.0e-04, eps_dual_inf = 1.0e-04,
          rho = 1.00e-01 (adaptiv

\anaconda3\envs\layers\lib\site-packages\cvxpy\expressions\expression.py:550: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.

  warnings.warn(__STAR_MATMUL_WARNING__, UserWarning)
\anaconda3\envs\layers\lib\site-packages\cvxpy\expressions\expression.py:550: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.

  warnings.warn(__STAR_MATMUL_WARNING__, UserWarning)
\anaconda3\envs\layers\lib\site-packages\cvxpy\expressions\expression.py

           OSQP v0.6.0  -  Operator Splitting QP Solver
              (c) Bartolomeo Stellato,  Goran Banjac
        University of Oxford  -  Stanford University 2019
-----------------------------------------------------------------
problem:  variables n = 4, constraints m = 4
          nnz(P) + nnz(A) = 10
settings: linear system solver = qdldl,
          eps_abs = 1.0e-05, eps_rel = 1.0e-05,
          eps_prim_inf = 1.0e-04, eps_dual_inf = 1.0e-04,
          rho = 1.00e-01 (adaptive),
          sigma = 1.00e-06, alpha = 1.60, max_iter = 10000
          check_termination: on (interval 25),
          scaling: on, scaled_termination: off
          warm start: on, polish: on, time_limit: off

iter   objective    pri res    dua res    rho        time
   1  -6.7562e-01   9.17e-01   4.80e+00   1.00e-01   1.04e-03s
  50  -4.7448e-01   2.31e-07   1.36e-06   2.10e+01   2.97e-03s
plsh  -4.7448e-01   1.49e-22   2.22e-16   --------   4.61e-03s

status:               solved
solution polish:      s

tensor([[[ 1.5812e-01,  2.8346e-01],
         [ 1.1116e-01,  3.1366e-02]],

        [[ 3.9089e-02, -2.2634e-01],
         [ 7.5505e-02, -2.1416e-01]],

        [[-2.9701e-02,  6.0879e-04],
         [-2.5919e-01, -8.1453e-02]],

        [[ 2.0373e-01, -4.5013e-01],
         [ 1.4177e-02, -1.5331e-02]],

        [[-2.3738e-02, -1.8325e-01],
         [-1.1157e-01, -2.1287e-01]],

        [[ 4.3565e-01, -5.0993e-01],
         [ 5.5012e-02, -2.8334e-01]],

        [[-6.5457e-02, -5.2836e-01],
         [ 1.0297e-01, -3.3117e-01]],

        [[-5.0423e-01,  5.7046e-01],
         [-3.6701e-01,  5.3042e-02]],

        [[-2.1234e-02,  8.3147e-02],
         [-1.0218e-01, -1.6603e-01]],

        [[-6.6644e-01, -9.7705e-03],
         [-2.5427e-01, -3.4575e-01]],

        [[-2.5351e-01,  2.5229e-01],
         [-4.1741e-01,  6.2519e-02]],

        [[ 6.9163e-02,  6.1926e-02],
         [-4.6791e-02,  3.0435e-02]],

        [[-4.3097e-01,  6.7309e-02],
         [-2.5239e-01, -3.4343e-01]],

        [[ 5

\anaconda3\envs\layers\lib\site-packages\cvxpy\expressions\expression.py:550: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.

  warnings.warn(__STAR_MATMUL_WARNING__, UserWarning)
\anaconda3\envs\layers\lib\site-packages\cvxpy\expressions\expression.py:550: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.

  warnings.warn(__STAR_MATMUL_WARNING__, UserWarning)
\anaconda3\envs\layers\lib\site-packages\cvxpy\expressions\expression.py

In [4]:
class DumbNet(nn.Module):
    def __init__(self, nFeatures, nHidden=20, nCls=2, nineq=2):
        super().__init__()
        self.fc1 = nn.Linear(nFeatures, nHidden)
        self.fc2 = nn.Linear(nHidden, nineq * nCls + nineq)
        
    def forward(self, x):
        # Normal FC network.
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
      
        return x


In [5]:
dumbnet = DumbNet(nFeatures)

for dumbparam, param in zip(dumbnet.parameters(), model.parameters()):
    print(param)
    if dumbparam.requires_grad:
        dumbparam.data = param.data


Parameter containing:
tensor([[-0.1066,  0.2092,  0.1837,  0.1319, -0.1404, -0.1222,  0.0325, -0.1082,
         -0.0021, -0.1164, -0.0869,  0.0171, -0.1881,  0.0475, -0.0128, -0.1034,
         -0.1218, -0.1242,  0.1702, -0.0015],
        [-0.0582, -0.0237, -0.1722, -0.1932, -0.1625, -0.0418, -0.0966,  0.0725,
          0.1443, -0.1549,  0.1347, -0.0536, -0.0857,  0.2063, -0.1339,  0.0670,
          0.0743,  0.0793, -0.1083,  0.2007],
        [-0.1506, -0.0875,  0.2335, -0.0410, -0.1009, -0.2128, -0.2322, -0.1914,
         -0.0544, -0.1580,  0.0813, -0.0655, -0.1132, -0.1324,  0.2129,  0.0208,
          0.0558,  0.0335,  0.2084, -0.0179],
        [-0.0504, -0.0707, -0.1568, -0.0488,  0.0122,  0.0466,  0.0469, -0.0083,
          0.1832,  0.0495, -0.0594, -0.1395, -0.1936, -0.1492,  0.1026,  0.2073,
         -0.0206,  0.1587, -0.0701,  0.0737],
        [ 0.1524, -0.1699, -0.0223, -0.0836, -0.0366,  0.1986, -0.1000,  0.1764,
          0.0055, -0.2021, -0.0139, -0.0404, -0.1430,  0.1871, -0

In [6]:
torch.onnx.export(dumbnet, (x[0]), 'exported_cvxpy.onnx', verbose=True)

graph(%input.1 : Float(20:1, requires_grad=0, device=cpu),
      %fc1.bias : Float(20:1, requires_grad=1, device=cpu),
      %fc2.bias : Float(6:1, requires_grad=1, device=cpu),
      %12 : Float(20:1, 20:20, requires_grad=0, device=cpu),
      %13 : Float(20:1, 6:20, requires_grad=0, device=cpu)):
  %6 : Float(20:1, requires_grad=1, device=cpu) = onnx::MatMul(%input.1, %12) # \anaconda3\envs\layers\lib\site-packages\torch\nn\functional.py:1692:0
  %7 : Float(20:1, requires_grad=1, device=cpu) = onnx::Add(%6, %fc1.bias)
  %8 : Float(20:1, requires_grad=1, device=cpu) = onnx::Relu(%7) # \anaconda3\envs\layers\lib\site-packages\torch\nn\functional.py:1136:0
  %10 : Float(6:1, requires_grad=1, device=cpu) = onnx::MatMul(%8, %13) # \anaconda3\envs\layers\lib\site-packages\torch\nn\functional.py:1692:0
  %11 : Float(6:1, requires_grad=1, device=cpu) = onnx::Add(%10, %fc2.bias)
  return (%11)



In [7]:
adv_x = torch.Tensor([0.0, 0.7022075686362291, 0.6525786846598369, 0.3066496669615028, 0.6556162906082897, 1.0, 0.0, 0.0, 1.0, 0.0, 0.11826336295490171, 0.11594182369836208, 0.040307531249216884, 1.0, 0.0, 0.0, 1.0, 0.8495636779011424, 0.0, 1.0])
adv_x = adv_x.reshape((1, 20))
model(adv_x)

tensor([[ 0.1000,  0.1000, -0.1000, -0.1000,  0.1000, -0.2000]],
       grad_fn=<AddmmBackward>)
-----------------------------------------------------------------
           OSQP v0.6.0  -  Operator Splitting QP Solver
              (c) Bartolomeo Stellato,  Goran Banjac
        University of Oxford  -  Stanford University 2019
-----------------------------------------------------------------
problem:  variables n = 4, constraints m = 4
          nnz(P) + nnz(A) = 10
settings: linear system solver = qdldl,
          eps_abs = 1.0e-05, eps_rel = 1.0e-05,
          eps_prim_inf = 1.0e-04, eps_dual_inf = 1.0e-04,
          rho = 1.00e-01 (adaptive),
          sigma = 1.00e-06, alpha = 1.60, max_iter = 10000
          check_termination: on (interval 25),
          scaling: on, scaled_termination: off
          warm start: on, polish: on, time_limit: off

iter   objective    pri res    dua res    rho        time
   1  -6.4760e-01   3.18e-01   2.00e+01   1.00e-01   1.43e-04s
  50   1.0000e+3

\anaconda3\envs\layers\lib\site-packages\cvxpy\expressions\expression.py:550: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.

  warnings.warn(__STAR_MATMUL_WARNING__, UserWarning)
\anaconda3\envs\layers\lib\site-packages\cvxpy\expressions\expression.py:550: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.

  warnings.warn(__STAR_MATMUL_WARNING__, UserWarning)


AssertionError: 